In [4]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [5]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [6]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [7]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

(43266, 10)


In [8]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

(18543, 9)


In [9]:
df = pd.concat([df_train,df_test],axis= 0)
print df.shape
df.head()

(61809, 10)


/home/prashant/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Company-response,Complaint-ID,Complaint-Status,Complaint-reason,Consumer-complaint-summary,Consumer-disputes,Date-received,Date-sent-to-company,Transaction-Type,tag
0,NaN,Tr-1,Closed with explanation,"Loan servicing, payments, escrow account","Seterus, Inc a déposé un faux rapport auprès d...",Yes,11/11/2015,11/11/2015,Mortgage,train
1,Company chooses not to provide a public response,Tr-2,Closed with non-monetary relief,Incorrect information on credit report,XX / XX / XXXX La requête en faillite n ° XXXX...,No,7/7/2015,7/7/2015,Credit reporting,train
2,NaN,Tr-3,Closed with explanation,Using a debit or ATM card,"El XXXX / XXXX / 15, estaba preparando el vuel...",No,5/7/2015,5/7/2015,Bank account or service,train
3,Company believes it acted appropriately as aut...,Tr-4,Closed with explanation,Cont'd attempts collect debt not owed,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",No,11/12/2016,11/12/2016,Debt collection,train
4,Company has responded to the consumer and the ...,Tr-5,Closed with explanation,Payoff process,J'ai obtenu un compte de crédit de soins pour ...,No,9/29/2016,9/29/2016,Credit card,train


In [10]:
df['Transaction-Type'] = df['Transaction-Type'].map(lambda x : x.lower().replace(' ','_'))

In [11]:
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [12]:
df["Complaint-reason"] = df["Complaint-reason"].map(str) + ' ' + df["Transaction-Type"]

In [13]:
text = list(df['Complaint-reason'])
vectorizer = TfidfVectorizer(min_df=1)
# tokenize and build vocab
vectorizer.fit(text)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [14]:
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
#print(vector.toarray())
vector

(61809, 219)


<61809x219 sparse matrix of type '<type 'numpy.float64'>'
	with 286566 stored elements in Compressed Sparse Row format>

In [15]:
vec_df = pd.DataFrame(vector.toarray())

In [16]:
print vec_df.shape
vec_df.head()


(61809, 219)


,0,1,2,3,4,5,6,7,8,9,...,209,210,211,212,213,214,215,216,217,218
0,0.0,0.0,0.0,0.0,0.0,0.0,0.386472,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
#vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])

In [18]:
#vec_df['cd'] = list(df['Consumer-disputes'])

In [19]:
#complete_df_one_hot = pd.get_dummies(data=vec_df, columns=['cd'])

In [20]:
complete_df_one_hot = vec_df

In [21]:
train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)


/home/prashant/.local/lib/python2.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
train_data.shape

(43266, 221)

In [23]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,Complaint-Status,Complaint-ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.386472,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Closed with explanation,Tr-1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Closed with non-monetary relief,Tr-2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Closed with explanation,Tr-3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Closed with explanation,Tr-4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Closed with explanation,Tr-5


In [24]:
y_train = (train_data['Complaint-Status'])
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [ ]:
# pd.concat([x_train,feature_tt])
x_train.shape

In [25]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

(43266,)
(43266, 5)


In [ ]:
x_train.head()

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0)


In [3]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
tpot = TPOTClassifier(generations = 8, population_size = 50, verbosity=2)
tpot.fit(X_train, y_train_internal)

/home/prashant/.local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
